In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
import random

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sS = pd.read_csv("sampleSubmission.csv")

In [3]:
select_label = [0, 1, 2, 3]
select_habits = [0., 1.]

In [4]:
train['推薦類型'].replace(['肌力',"柔軟度","心肺","肌耐力"],[0,1,2,3] , inplace = True)

In [5]:
train = train[train['推薦類型'].isin(select_label)]
train = train[train['拳擊'].isin(select_habits)]
train = train[train['器材'].isin(select_habits)]
train = train[train['有氧'].isin(select_habits)]
train = train[train['舞蹈'].isin(select_habits)]
train = train[train['高強度間歇'].isin(select_habits)]
train = train[train['瑜伽'].isin(select_habits)]
train = train[train['肌力訓練'].isin(select_habits)]
train = train[train['燃脂'].isin(select_habits)]
train = train[train['減脂瘦身'].isin(select_habits)]
train = train[train['增加肌肉'].isin(select_habits)]
train = train[train['增強體魄'].isin(select_habits)]
train = train[train['舒緩壓力'].isin(select_habits)]
train = train[train['年齡']<70]
train = train[train['BMI']>10]

In [6]:
train['推薦類型'] = train['推薦類型'].astype("int64")
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3427 entries, 5 to 26416
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   性別      3427 non-null   float64
 1   年齡      3427 non-null   float64
 2   BMI     3427 non-null   float64
 3   拳擊      3427 non-null   float64
 4   器材      3427 non-null   float64
 5   有氧      3427 non-null   float64
 6   舞蹈      3427 non-null   float64
 7   高強度間歇   3427 non-null   float64
 8   瑜伽      3427 non-null   float64
 9   肌力訓練    3427 non-null   float64
 10  燃脂      3427 non-null   float64
 11  減脂瘦身    3427 non-null   float64
 12  增加肌肉    3427 non-null   float64
 13  增強體魄    3427 non-null   float64
 14  舒緩壓力    3427 non-null   float64
 15  推薦類型    3427 non-null   int64  
dtypes: float64(15), int64(1)
memory usage: 455.1 KB


In [7]:
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE,BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
X = train.drop(["推薦類型"], axis =1)
y = train["推薦類型"]

scaler = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled)
X_scaled = X_scaled.rename(columns = {0:"性別",1:'年齡',2:'BMI',3:'拳擊',4:'器材',5:'有氧',6:'舞蹈',7:'高強度間歇',8:'瑜伽',9:'肌力訓練',10:'燃脂',11:'減脂瘦身',12:'增加肌肉',13:'增強體魄',14:'舒緩壓力', 15:'label'})
#X_s, y_s =  TomekLinks().fit_resample(X_scaled, y)
X_s, y_s = BorderlineSMOTE(random_state = 0).fit_resample(X_scaled, y)

X_train, X_val, y_train, y_val = train_test_split(
   X_s, y_s, test_size=0.33, random_state=42)

In [8]:
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
SV = SVC(C = 100,gamma =1.8)
SV.fit(X_train,y_train)
SV.score(X_val,y_val)

0.8569636135508155

In [9]:
X_scaled["年齡"].value_counts()

0.648148    352
0.666667    294
0.296296    175
0.629630    167
0.407407    148
0.685185    146
0.611111    142
0.370370    130
0.574074    112
0.500000     98
0.592593     95
0.351852     94
0.333333     93
0.277778     89
0.425926     85
0.555556     84
0.481481     84
0.462963     83
0.314815     80
0.444444     79
0.518519     71
0.537037     69
0.388889     64
0.722222     63
0.222222     62
0.240741     58
0.203704     57
0.703704     47
0.259259     45
0.148148     20
0.092593     19
0.129630     17
0.925926     16
0.833333     16
0.166667     15
0.037037     15
0.074074     13
0.777778     12
0.796296     11
0.740741     11
0.111111     10
0.759259     10
0.814815      9
0.870370      9
0.185185      9
0.018519      8
0.055556      8
0.851852      7
0.000000      7
0.944444      5
0.981481      4
0.907407      4
0.888889      4
0.962963      1
1.000000      1
Name: 年齡, dtype: int64

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(test)
test = scaler.transform(test)
test = pd.DataFrame(test)




SV.fit(X_s,y_s)
y_pred = SV.predict(test)
y_pred_pd = pd.DataFrame(data = y_pred, columns = ["label"])
y_pred_pd = y_pred_pd.astype('int64').reset_index()
y_pred_pd
y_pred_pd.to_csv("e94084032.csv", index = False)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [11]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

def accuracy_score(y_val,y_pred):
    Sum = 0
    y_val = y_val.tolist()
    y_pred = y_pred.tolist()
    for i in range(len(y_pred)):
        if y_val[i] == y_pred[i]:
            Sum+=1
    score = Sum/len(y_pred)
    
    return score


rnd_clf = RandomForestClassifier(random_state=100,n_estimators=700)
xgb_clf =  XGBClassifier(random_state=100, n_estimators=300,learning_rate=0.15)
svc_clf = SVC(C= 80, gamma = 1.7)

voting_clf = VotingClassifier(estimators=[('rf', rnd_clf), ('sv', svc_clf),('xg',xgb_clf)],voting='hard')
scores_1 = cross_val_score(rnd_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores_2 = cross_val_score(xgb_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores_3 = cross_val_score(svc_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
scores = cross_val_score(voting_clf,X_s,y_s,cv = 3,scoring = 'accuracy')
print("rnd: "+str(scores_1.mean()))
print(" xgb: "+str(scores_2.mean()))
print(" svc: "+str(scores_3.mean()))
print(" vote: "+str(scores.mean()))

KeyboardInterrupt: 

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(test)
test = scaler.transform(test)
test = pd.DataFrame(test)




voting_clf.fit(X_s,y_s)
y_pred = voting_clf.predict(test)
y_pred_pd = pd.DataFrame(data = y_pred, columns = ["label"])
y_pred_pd = y_pred_pd.astype('int64').reset_index()
y_pred_pd
y_pred_pd.to_csv("e94084032.csv", index = False)

NameError: name 'df' is not defined